In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
from datetime import datetime
import re

import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import pytz


%matplotlib inline

In [2]:
import psutil
print( "{0:17s}{1:} CPUs PHYSICAL".format( "psutil:", psutil.cpu_count( logical = False ) ) )
print( "{0:17s}{1:} CPUs LOGICAL".format( "psutil:", psutil.cpu_count( logical = True  ) ) )

psutil:          8 CPUs PHYSICAL
psutil:          8 CPUs LOGICAL


In [231]:
def format_cols_to(df, cols, dtype = 'float64'):
    """Format a list of columns to desired
    datatype.

    Args:
        df (pd.DataFrame): input dataframe
        cols (list): columns list
        dtype (str, optional): Data type. Defaults to 'float64'.

    Returns:
        _type_: _description_
    """

    for col in cols:
            df.loc[:,col] = df[col].astype(dtype)

    return df

def reindex_by_date(df):
    """
    Reindex Time Series. 
    """
    dates = range(df.index[0], df.index[-1]+60000,60000)
    return df.reindex(dates, method = 'pad')


def todatetime(t):
    """
    Convert Unix timestamp to datetime.
    """
    return datetime.fromtimestamp(t/1000)

input_folder_db = '/mnt/Data/Tensor_Invest_Fund/data/'
input_folder_db = '/mnt/Data/Tensor_Invest_Fund/data/Cryptos/'


SYMBOLS = ['BNBUSDT', 'BNBBTC', 'BTCUSDT', 'EOSUSDT', 'ETCUSDT',
       'LTCUSDT', 'XMRBTC', 'TRXUSDT', 'XLMUSDT', 'ADAUSDT', 'IOTAUSDT',
       'MKRUSDT', 'DOGEUSDT','ETHUSDT']


initial_date_load = datetime(2021,9,1)

years_filter = [2021, 2022]


years_filter = [2022]

In [174]:
# Fix duplicates

for symbol in SYMBOLS[0:1]:
    for year in years_filter:

        #file_name = os.path.join(input_folder_db, f'{folder_path}{stock}', f'{year}{parquet_file_name}{stock}.parquet'  )
        file_name = os.path.join(input_folder_db, f'{symbol}', f'{year}_{symbol}.parquet'  )

        print("Reading file", file_name)
        
        df = pd.read_parquet(file_name)

        if False:

            #df = df[df['Date'] >= initial_date_load].copy()

            cols = ['Open', 'High', 'Low', 'Close', 'Volume', 
        'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume',
        'Taker Buy Quote Asset Volume']
            df = format_cols_to(df, cols, dtype = 'float64')

            cols = ['Number of Trades', 'Year']
            df = format_cols_to(df, cols, dtype = 'int32')

            df.loc[:,'max_trades'] = df.groupby('Open Time')['Number of Trades'].transform(np.max)

            df = df[df['Number of Trades'] == df['max_trades']].copy().drop_duplicates().reset_index(drop = True)

            assert df[df.duplicated()].empty, "Duplicates"

            #break
            print(" Storing file", file_name)
            df.to_parquet(file_name)

        #del df


Reading file /mnt/Data/Tensor_Invest_Fund/data/Cryptos/BNBUSDT/2022_BNBUSDT.parquet


In [186]:
data 

,Open Time,Open,High,Low,Close,Volume,Clos Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,Date,Year,Ticker,max_trades
timestamp,,,,,,,,,,,,,,,,
1640995200000,1640995200000,511.5,512.5,511.4,512.2,423.922,1640995259999,2.170870e+05,303,293.953,150544.2531,0,2022-01-01 00:00:00,2022,BNBUSDT,303
1640995260000,1640995260000,512.2,513.3,511.7,513.1,709.690,1640995319999,3.637906e+05,374,345.070,176824.7579,0,2022-01-01 00:01:00,2022,BNBUSDT,374
1640995320000,1640995320000,513.1,513.6,512.8,513.5,482.053,1640995379999,2.474258e+05,318,163.958,84161.2410,0,2022-01-01 00:02:00,2022,BNBUSDT,318
1640995380000,1640995380000,513.5,513.7,512.8,512.9,459.408,1640995439999,2.358014e+05,264,177.987,91377.4080,0,2022-01-01 00:03:00,2022,BNBUSDT,264
1640995440000,1640995440000,513.0,513.2,512.7,512.9,277.571,1640995499999,1.423621e+05,182,119.327,61211.6888,0,2022-01-01 00:04:00,2022,BNBUSDT,182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1648221000000,1648221000000,416.6,416.7,416.2,416.5,670.311,1648221059999,2.791439e+05,310,192.324,80093.3372,0,2022-03-25 15:10:00,2022,BNBUSDT,310
1648221060000,1648221060000,416.6,416.6,416.4,416.5,209.369,1648221119999,8.721245e+04,188,131.475,54769.6723,0,2022-03-25 15:11:00,2022,BNBUSDT,188
1648221120000,1648221120000,416.6,416.6,416.2,416.2,239.928,1648221179999,9.991350e+04,223,26.608,11083.3920,0,2022-03-25 15:12:00,2022,BNBUSDT,223


In [233]:
data = df.copy()
data.loc[:,'timestamp'] = data['Open Time'].copy()
data.set_index('timestamp', inplace = True)
data.sort_values(by = ['timestamp'], inplace = True)

In [234]:
dates = range(data.index[0], data.index[-1]+60000,60000)
data = data.reindex(dates, method = 'pad').reset_index()

In [235]:
data.head()

,timestamp,Open Time,Open,High,Low,Close,Volume,Clos Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,Date,Year,Ticker,max_trades
0,1640995200000,1640995200000,511.5,512.5,511.4,512.2,423.922,1640995259999,217086.9816,303,293.953,150544.2531,0,2022-01-01 00:00:00,2022,BNBUSDT,303
1,1640995260000,1640995260000,512.2,513.3,511.7,513.1,709.690,1640995319999,363790.6030,374,345.070,176824.7579,0,2022-01-01 00:01:00,2022,BNBUSDT,374
2,1640995320000,1640995320000,513.1,513.6,512.8,513.5,482.053,1640995379999,247425.8464,318,163.958,84161.2410,0,2022-01-01 00:02:00,2022,BNBUSDT,318
3,1640995380000,1640995380000,513.5,513.7,512.8,512.9,459.408,1640995439999,235801.4373,264,177.987,91377.4080,0,2022-01-01 00:03:00,2022,BNBUSDT,264
4,1640995440000,1640995440000,513.0,513.2,512.7,512.9,277.571,1640995499999,142362.0936,182,119.327,61211.6888,0,2022-01-01 00:04:00,2022,BNBUSDT,182


In [237]:
# Convert Unix timestamp to datetime 
data.loc[:,'timestamp'] = data['timestamp'].apply(todatetime)   

data.loc[:,'timestamp'] = pd.to_datetime(data['timestamp'], unit = "ms")

In [239]:
data[~(data['timestamp'] == data['Date'])]['timestamp']

0        2022-01-01 01:00:00+01:00
1        2022-01-01 01:01:00+01:00
2        2022-01-01 01:02:00+01:00
3        2022-01-01 01:03:00+01:00
4        2022-01-01 01:04:00+01:00
                    ...           
120430   2022-03-25 16:10:00+01:00
120431   2022-03-25 16:11:00+01:00
120432   2022-03-25 16:12:00+01:00
120433   2022-03-25 16:13:00+01:00
120434   2022-03-25 16:14:00+01:00
Name: timestamp, Length: 120435, dtype: datetime64[ns, Europe/Berlin]

In [178]:
data = (data.set_index('timestamp')
        .groupby(['Ticker'], 
        group_keys= False)
        .apply(reindex_by_date)
        .reset_index())

# Convert Unix timestamp to datetime 
data['timestamp'] = data['timestamp'].apply(todatetime)   

ValueError: index must be monotonic increasing or decreasing

In [144]:
dfs = []

for symbol in SYMBOLS[0:1]:
    for year in years_filter:

        #file_name = os.path.join(input_folder_db, f'{folder_path}{stock}', f'{year}{parquet_file_name}{stock}.parquet'  )
        file_name = os.path.join(input_folder_db, f'{symbol}', f'{year}_{symbol}.parquet'  )

        print("Reading file", file_name)
        
        df = pd.read_parquet(file_name)

        df = df[df['Date'] >= initial_date_load].copy()

        cols = ['Open', 'High', 'Low', 'Close', 'Volume', 
       'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume',
       'Taker Buy Quote Asset Volume']
        df = format_cols_to(df, cols, dtype = 'float64')

        cols = ['Number of Trades', 'Year']
        df = format_cols_to(df, cols, dtype = 'int32')

        dfs.append(df)

        del df

data = pd.concat(dfs, ignore_index = True).drop_duplicates()


data.loc[:,'timestamp'] = data['Open Time'].copy()

data = (data.set_index('timestamp')
        .groupby(['Ticker'], 
        group_keys= False)
        .apply(reindex_by_date)
        .reset_index())

# Convert Unix timestamp to datetime 
data['timestamp'] = data['timestamp'].apply(todatetime)  


Reading file /mnt/Data/Tensor_Invest_Fund/data/Cryptos/BNBUSDT/2021_BNBUSDT.parquet
Reading file /mnt/Data/Tensor_Invest_Fund/data/Cryptos/BNBUSDT/2022_BNBUSDT.parquet


ValueError: cannot reindex a non-unique index with a method or limit

In [147]:
data[data['Open Time'] == 1648163880000]

,Open Time,Open,High,Low,Close,Volume,Clos Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,Date,Year,Ticker,timestamp
175560,1648163880000,413.4,413.5,413.2,413.2,473.138,1648163939999,195586.9416,134,18.407,7609.6973,0,2022-03-24 23:18:00,2022,BNBUSDT,1648163880000
179466,1648163880000,413.4,413.5,413.2,413.3,452.311,1648163939999,186980.8272,101,14.427,5964.7633,0,2022-03-24 23:18:00,2022,BNBUSDT,1648163880000


In [148]:
data[data['Open Time'] == 1647986940000]

,Open Time,Open,High,Low,Close,Volume,Clos Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,Date,Year,Ticker,timestamp
176517,1647986940000,406.0,406.0,405.7,405.8,616.947,1647986999999,250390.5250,185,139.775,56731.6487,0,2022-03-22 22:09:00,2022,BNBUSDT,1647986940000
181124,1647986940000,406.0,406.0,405.8,405.8,489.063,1647986999999,198501.1963,95,71.876,29178.2345,0,2022-03-22 22:09:00,2022,BNBUSDT,1647986940000


In [146]:
data[data['Open Time'].duplicated()]

,Open Time,Open,High,Low,Close,Volume,Clos Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,Date,Year,Ticker,timestamp
179466,1648163880000,413.4,413.5,413.2,413.3,452.311,1648163939999,186980.8272,101,14.427,5964.7633,0,2022-03-24 23:18:00,2022,BNBUSDT,1648163880000
181124,1647986940000,406.0,406.0,405.8,405.8,489.063,1647986999999,198501.1963,95,71.876,29178.2345,0,2022-03-22 22:09:00,2022,BNBUSDT,1647986940000
199586,1647887520000,397.3,397.7,397.2,397.7,93.265,1647887579999,37062.5339,102,71.547,28432.5523,0,2022-03-21 18:32:00,2022,BNBUSDT,1647887520000


# Triple Barrier Method


**FN**: False Negatives. (missed oportunity)
**TP**: True Positive. (win money)
**TN**: True Negative. (avoids losing money)
**FP**: False Positive. (lose money)

- **type-I error** : False Positive
- **type-II errorr**: False Negative
- **Precision**: TP/ (TP + FP)
- **Recall**: TP/ (TP + FN)

### Meta Labeling

- **Model1**: high Recall, no particularly high Precision. Identify all positives, at the expense of false positives. This can be a foundamental model like crossing moving averages, or mean-reverting strategy on Bollinger Bands, or just an ML Model itself.
- **Model2**: Correction of low Precision. Determine whether a positive from the primariy model is true or false.

Meta-labeling allows you to build an ML system on top of a white box.

In [132]:
class TripleBarrierMethod():
  """Calculate Triple Barrier Labels based
    on profit target and stop loss strategy.
    As well as dynamic volatility as variable
    reference for selection of targeted horizontal
    barriers.

    Excepted input is pandas dataframe with 
    candle bars data of the form
    OHLC. Index are timestamps. 

    Run method generates a Dataframe containing 
    the a label for every row (candle bar). 

    1: Profitable trade
    0: No loss no gain
    -1: Stop loss

    Ref: https://towardsdatascience.com/financial-machine-learning-part-1-labels-7eeed050f32e
  """


  def __init__(self, data, ticker, ptsl, delta_vertical_b, pt, 
                  delta_volatility = pd.Timedelta(hours=1), 
                  span_volatility = 100, 
                  n_jobs = 7,
                  parallel_calculation = True, 
                  verbose = 0):
    """Intializer method

    Args:
        data (pd.DataFrame): dataframe of the form OHLC.
        ticker (str): ticker
        ptsl (list): Profit-Stop Loss ratio [profit, stop]
        delta_vertical_b (pd.Timedelta): Trade holding time. 
        pt (int): Position Type. 1: Long. -1: Short
        delta_volatility (pd.Timedelta, optional): delta for volatility calculation. 
                                                    Defaults to pd.Timedelta(hours=1).
        span_volatility (int, optional): Specify decay in terms of span. Defaults to 100.
        n_jobs (int, optional): number of parallel jobs
    """

    self._data = data
    self.ticker = ticker
    self.ptsl = ptsl
    self.delta_vertical_b = delta_vertical_b
    self.pt = pt
    self.delta_volatility = delta_volatility
    self.span_volatility = span_volatility
    self._h_barriers = None
    self.n_jobs = n_jobs
    self.parallel_calculation = parallel_calculation
    self.verbose = verbose

  @property
  def data(self):
    return self._data

  @property
  def h_barriers(self):
    return self._h_barriers

  def run(self):
    """Run strategy label calculation. 

    Returns:
        pd.DataFrame: Dataframe containing the TBM label
                      for every row (candle bar). 
    """


    # Hourly Volatility
    self._data =  self._data.assign(threshold = self.calculate_volatility()).dropna()

    # Get vertical barriers timestamp
    self._data = self._data.assign(t1 = self.calculate_horizons()).dropna()

    # Get Barriers's timestamps
    events = self._data[['t1', 'threshold']].copy()

    # Add side
    events = events.assign(side=pd.Series(self.pt , events.index)) 

    # Get horizontal barriers (Target profil and stop loss)
    self._h_barriers = self.calculate_h_barriers(events)

    # Get time of barrier touches
    if self.parallel_calculation:
      touches = self.calculate_touches_parallel(events)
    else:
      touches = self.calculate_touches(events)

    # Label Barrier touches
    if self.parallel_calculation:
      touches = self.calculate_labels_parallel(touches)
    else:
      touches = self.calculate_labels(touches)

    # Add labels to data
    self._data = self._data.assign(label=touches.label.astype(int))


  def calculate_volatility(self):
      """Calculate volatility using
      exponentially weighted standard deviation. 
 
      Returns:
          pd.Series: ewm Standard deviation
      """

      print("Getting volatility")

      # 1. compute returns of the form p[t]/p[t-1] - 1
      close = self._data.Close

      # 1.1 find the timestamps of p[t-1] values
      df0 = close.index.searchsorted(close.index-self.delta_volatility)
      df0 = df0[df0>0]

      # 1.1 find the timestamps of p[t-1] values
      df0 = pd.Series(close.index[df0-1], index = close.index[close.shape[0]-df0.shape[0]:])

      # 1.3 get values by timestamps, then compute returns
      df0 = close.loc[df0.index]/close.loc[df0.values].values - 1 

      # 2. estimate rolling standard deviation
      df0 = df0.ewm(span =  self.span_volatility).std()

      return df0

  def calculate_horizons(self):
      """Get vertical barriers timestamp.

      Returns:
          pd.Series: pandas Series with vertical timestamps barriers
      """
      print("Getting horizons")

      close = self._data.Close
      t1 = close.index.searchsorted(close.index + self.delta_vertical_b)
      t1 = t1[t1 < close.shape[0]]
      t1 = close.index[t1]
      return pd.Series(t1, index=close.index[:t1.shape[0]])


  def calculate_h_barriers(self, events):
    """Get horizontal barriers

    Args:
        events: pd dataframe with columns
          t1: timestamp of the next horizon
          threshold: unit height of top and bottom barriers
          side: the side of each bet
          factors: multipliers of the threshold to set the height of 
                  top/bottom barriers

    Returns:
        pd.DataFrame: df containing the upper and lower horizontal
                      barriers
    """

    if self.ptsl[0] > 0: 
      thresh_uppr = self.ptsl[0] * events['threshold']
    else: 
      thresh_uppr = pd.Series(index=events.index) # no uppr thresh

    if self.ptsl[1] > 0: 
      thresh_lwr = -self.ptsl[1] * events['threshold']
    else: 
      thresh_lwr = pd.Series(index=events.index)  # no lwr thresh  

    return pd.DataFrame({'thresh_uppr':thresh_uppr,  \
                        'thresh_lwr':thresh_lwr},  \
                        index=events.index)

     

  def calculate_touches(self, events):
    '''
    events: pd dataframe with columns
      t1: timestamp of the next horizon
      threshold: unit height of top and bottom barriers
      side: the side of each bet
      factors: multipliers of the threshold to set the height of 
              top/bottom barriers

    Returns:
          pd.Series: time of earliest profit or stop loss 
    '''  
    print("Getting touches")

    thresh_lwr = self._h_barriers['thresh_lwr'].copy()
    thresh_uppr = self._h_barriers['thresh_uppr'].copy()

    out = events[['t1']].copy(deep=True)

    for loc, t1 in events['t1'].iteritems():

      df0 = self._data.Close[loc:t1]                              # path prices
      df0 = (df0 / self._data.Close[loc] - 1) * events.side[loc]  # path returns

      out.loc[loc, 'stop_loss'] = df0[df0 < thresh_lwr[loc]].index.min()  # earliest stop loss
      out.loc[loc, 'take_profit'] = df0[df0 > thresh_uppr[loc]].index.min() # earliest take profit
    return out


  def calculate_touches_parallel(self, events):
    '''
    events: pd dataframe with columns
      t1: timestamp of the next horizon
      threshold: unit height of top and bottom barriers
      side: the side of each bet
      factors: multipliers of the threshold to set the height of 
              top/bottom barriers

    Returns:
          pd.Series: time of earliest profit or stop loss 
    '''  
    print("Getting touches")

    thresh_lwr = self._h_barriers['thresh_lwr'].copy()
    thresh_uppr = self._h_barriers['thresh_uppr'].copy()

    Close = self._data.Close

    def parallel_touch(Close, events, loc, t1):
      """_summary_

      Args:
          Close (pd.Series): Close prices
          events (pd.DataFrame): events data frame
          loc (datetime.datetime): candle timestamp 
          t1 (datetime.datetime): candle timestamp + horizotal barrier offset

      Returns:
          pd.DataFrame: time of earliest touch for stop and profit
      """

      out = events.loc[[loc]].filter(['t1'])
      
      df0 = Close[loc:t1]                              # path prices
      df0 = (df0 / Close[loc] - 1) * events.side[loc]  # path returns

      out['stop_loss'] = df0[df0 < thresh_lwr[loc]].index.min()  # earliest stop loss
      out['take_profit'] = df0[df0 > thresh_uppr[loc]].index.min() # earliest take profit

      return out

    parallel_outputs = Parallel(n_jobs = self.n_jobs, verbose = self.verbose)(delayed(parallel_touch) \
                      (Close, events, loc, t1) for loc, t1 in events['t1'].iteritems())

    self.parallel_outputs = parallel_outputs

    return pd.concat(parallel_outputs)

  def calculate_labels(self, touches):
    """Assign TBM Labels

    Args:
        touches (pd.DataFrame): dataframe containing the 
            time of earliest profit or stop loss 

    Returns:
        pd.DataFrame: dataframe containing the labels
    """
    print("Getting Labels")

    out = touches.copy(deep=True)
    first_touch = touches[['stop_loss', 'take_profit']].min(axis=1)
    for loc, t in first_touch.iteritems():
      #print(" ", loc, " ", t)
      if pd.isnull(t):
        out.loc[loc, 'label'] = 0
      elif t == touches.loc[loc, 'stop_loss']: 
        out.loc[loc, 'label'] = -1
      else:
        out.loc[loc, 'label'] = 1
    return out

  def calculate_labels_parallel(self, touches):
    """Assign TBM Labels

    Args:
        touches (pd.DataFrame): dataframe containing the 
            time of earliest profit or stop loss 

    Returns:
        pd.DataFrame: dataframe containing the labels
    """
    print("Getting Labels")


    #out = touches.copy(deep=True)
    first_touch = touches[['stop_loss', 'take_profit']].min(axis=1)

    def parellel_labels(touches, loc, t):
      """Get labels for first horizontal
      barrier touch.


      Args:
          touches (pd.DataFrame): dataframe containing the 
            time of earliest profit or stop loss 
          loc (datetime.datetime): candle bar timestamp
          t (datetime.datetime): time of touch

      Returns:
          _type_: _description_
      """

      out = touches.loc[[loc]]

      if pd.isnull(t):
        out.loc[loc, 'label'] = int(0)
      elif t == touches.loc[loc, 'stop_loss']: 
        out.loc[loc, 'label'] = int(-1)
      else:
        out.loc[loc, 'label'] = int(1)

      return out

    parallel_outputs = Parallel(n_jobs = self.n_jobs, verbose = self.verbose)(delayed(parellel_labels) \
                      (touches, loc, t) for loc, t in first_touch.iteritems())

    return pd.concat(parallel_outputs)
    
  def add_metalabel(self, y_model1):
    """Calculate Metalabels

    Args:
        y_model1 (pd.Series): Trading Strategy containing
                          1: Long trades
                          0: No trade
                          -1: Short trades

    Returns:
        np.array: metalabels:
          1: Take the trade
          0: ignore the trade
    """

    y_true = self._data['label']
    
    bin_label = np.zeros_like(y_model1)
    for i in range(y_model1.shape[0]):
        if y_model1[i] != 0 and y_model1[i]*y_true[i] > 0:
            bin_label[i] = 1  # true positive

    self._data.loc[:,'metalabel'] =  bin_label

    return self._data

  def store_data(self, output_folder_db, v_barrier_minutes):
    """_summary_

    Args:
        output_folder_db (_type_): _description_
        v_barrier_minutes (_type_): _description_
    """

    file_name = 'Tripe_Barrier_Method_{}_ptsl_{}_vb_{}m.parquet'.format(self.ticker, '-'.join([str(x) for x in self.ptsl] ), v_barrier_minutes )
    _dir = os.path.join(output_folder_db, self.ticker)

    if not os.path.exists(_dir):
      os.mkdir(_dir)

    output_storage = os.path.join(_dir, file_name)

    print(output_storage)

    self._data.to_parquet(output_storage)


In [133]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5056345 entries, 0 to 5056344
Data columns (total 15 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   Open Time                     int64         
 1   Open                          float64       
 2   High                          float64       
 3   Low                           float64       
 4   Close                         float64       
 5   Volume                        float64       
 6   Clos Time                     int64         
 7   Quote Asset Volume            float64       
 8   Number of Trades              int32         
 9   Taker Buy Base Asset Volume   float64       
 10  Taker Buy Quote Asset Volume  float64       
 11  Ignore                        object        
 12  Date                          datetime64[ns]
 13  Year                          int32         
 14  Ticker                        object        
dtypes: datetime64[ns](1), float64(8)

In [134]:
SYMBOLS = ['BNBUSDT', 'BNBBTC', 'BTCUSDT', 'EOSUSDT', 'ETCUSDT',
       'LTCUSDT', 'XMRBTC', 'TRXUSDT', 'XLMUSDT', 'ADAUSDT', 'IOTAUSDT',
       'MKRUSDT', 'DOGEUSDT','ETHUSDT']

# Profit-Stop Loss ratio
ptsl = [1,1]
v_barrier_minutes = 15
delta_vertical_b = pd.Timedelta(minutes = v_barrier_minutes) # Vertical barrier length 
output_folder_db = '/mnt/Data/Tensor_Invest_Fund/data/Cryptos/TBM'

# Volatility Parameters
delta_volatility = pd.Timedelta(hours=1)
span_volatility = 100

# Position Type 
# 1: Long
# -1: Short
pt = 1
n_jobs = 8


for ticker in SYMBOLS:

    df_sub = data[(data['Date'] >'2022-03-25') & (data['Ticker'] == ticker)].copy()

    print("Generating TBM Labels for ticker", ticker)
    print(" in the timeframe ", df_sub['Date'].min(), " - ", df_sub['Date'].max())
    
    print(df_sub.shape)

    df_sub = df_sub.rename(columns = {'Date':'timestamp'}).set_index('timestamp').copy()


    TBM_parallel = TripleBarrierMethod(df_sub, 
                        ticker,
                        ptsl, 
                        delta_vertical_b, 
                        pt, 
                        delta_volatility, 
                        span_volatility, 
                        n_jobs,
                        parallel_calculation= True)

    TBM_parallel.run()

    print(TBM_parallel.data['label'].value_counts())

    TBM_parallel.store_data(output_folder_db, v_barrier_minutes)

    break

Generating TBM Labels for ticker BNBUSDT
 in the timeframe  2022-03-25 00:01:00  -  2022-03-25 15:14:00
(914, 15)
Getting volatility
Getting horizons
Getting touches
Getting Labels
 0    640
 1    101
-1     96
Name: label, dtype: int64
/mnt/Data/Tensor_Invest_Fund/data/Cryptos/TBM/BNBUSDT/Tripe_Barrier_Method_BNBUSDT_ptsl_1-1_vb_15m.parquet


In [84]:
df_tbm_p['label'].value_counts()

 0    1025
 1     171
-1     125
Name: label, dtype: int64

In [111]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 914 entries, 2022-03-25 00:01:00 to 2022-03-25 15:14:00
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Open Time                     914 non-null    int64 
 1   Open                          914 non-null    object
 2   High                          914 non-null    object
 3   Low                           914 non-null    object
 4   Close                         914 non-null    object
 5   Volume                        914 non-null    object
 6   Clos Time                     914 non-null    int64 
 7   Quote Asset Volume            914 non-null    object
 8   Number of Trades              914 non-null    int64 
 9   Taker Buy Base Asset Volume   914 non-null    object
 10  Taker Buy Quote Asset Volume  914 non-null    object
 11  Ignore                        914 non-null    object
 12  Year                          914 non-nul

In [64]:
df_sub.columns



cols = ['Number of Trades']


Index(['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Clos Time',
       'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume',
       'Taker Buy Quote Asset Volume', 'Ignore', 'Year', 'Ticker'],
      dtype='object')

In [65]:
df_sub.head()

,Open Time,Open,High,Low,Close,Volume,Clos Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,Year,Ticker
timestamp,,,,,,,,,,,,,,
2022-03-24 00:01:00,1648080060000,408.70000000,408.90000000,408.50000000,408.80000000,593.94500000,1648080119999,242732.22740000,288,372.58300000,152274.45340000,0,2022,BNBUSDT
2022-03-24 00:02:00,1648080120000,408.80000000,410.40000000,408.70000000,410.30000000,5888.75400000,1648080179999,2413522.14740000,2056,3922.02700000,1607117.14500000,0,2022,BNBUSDT
2022-03-24 00:03:00,1648080180000,410.20000000,410.40000000,409.70000000,409.80000000,1194.01200000,1648080239999,489523.08600000,560,362.55100000,148645.11920000,0,2022,BNBUSDT
2022-03-24 00:04:00,1648080240000,409.80000000,409.80000000,409.30000000,409.60000000,996.40200000,1648080299999,408009.07010000,408,506.37600000,207356.73330000,0,2022,BNBUSDT
2022-03-24 00:05:00,1648080300000,409.70000000,409.70000000,408.60000000,408.70000000,588.00800000,1648080359999,240556.99800000,511,135.69800000,55503.85190000,0,2022,BNBUSDT


In [62]:
df_sub.head()

,Open Time,Open,High,Low,Close,Volume,Clos Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,Year,Ticker
timestamp,,,,,,,,,,,,,,
2022-03-24 00:01:00,1648080060000,408.70000000,408.90000000,408.50000000,408.80000000,593.94500000,1648080119999,242732.22740000,288,372.58300000,152274.45340000,0,2022,BNBUSDT
2022-03-24 00:02:00,1648080120000,408.80000000,410.40000000,408.70000000,410.30000000,5888.75400000,1648080179999,2413522.14740000,2056,3922.02700000,1607117.14500000,0,2022,BNBUSDT
2022-03-24 00:03:00,1648080180000,410.20000000,410.40000000,409.70000000,409.80000000,1194.01200000,1648080239999,489523.08600000,560,362.55100000,148645.11920000,0,2022,BNBUSDT
2022-03-24 00:04:00,1648080240000,409.80000000,409.80000000,409.30000000,409.60000000,996.40200000,1648080299999,408009.07010000,408,506.37600000,207356.73330000,0,2022,BNBUSDT
2022-03-24 00:05:00,1648080300000,409.70000000,409.70000000,408.60000000,408.70000000,588.00800000,1648080359999,240556.99800000,511,135.69800000,55503.85190000,0,2022,BNBUSDT


In [34]:
TBM_linear = TripleBarrierMethod(df_sub, 
                            ptsl, 
                            delta_vertical_b, 
                            pt, 
                            delta_volatility, 
                            span_volatility, 
                            n_jobs,
                            parallel_calculation= False)

df_tbm_l = TBM_linear.run()

Getting volatility
Getting horizons
Getting touches
Getting Labels


In [24]:
df_tbm_p['label'].value_counts()

 0.0    4002
-1.0     756
 1.0     685
Name: label, dtype: int64

In [ ]:
df_tbm_l['label'].value_counts()

 0.0    7009
 1.0    1311
-1.0    1263
Name: label, dtype: int64

In [81]:
target_stop = TBM.get_targets_stop()

In [82]:
target_stop.head()

,thresh_uppr,thresh_lwr
timestamp,,
2022-01-31 01:03:00,0.001157,-0.001157
2022-01-31 01:04:00,0.000924,-0.000924
2022-01-31 01:05:00,0.000922,-0.000922
2022-01-31 01:06:00,0.000835,-0.000835
2022-01-31 01:07:00,0.000748,-0.000748


In [83]:
df_tbm['label'].value_counts()


 0.0    8068
 1.0    1518
-1.0    1377
Name: label, dtype: int64

In [68]:
# Hourly Volatility
data =  data.assign(threshold = calculate_volatility(data.Close)).dropna()

# Get vertical barriers timestamp
data = data.assign(t1 = calculate_horizons(data , delta)).dropna()

# Get Barriers's timestamps
events = data[['t1', 'threshold']] 

# Add side
events = events.assign(side=pd.Series(pt , events.index)) # long only

# Get time of barrier touches
touches = get_touches(data.Close, events, ptsl)

# Label Barrier touches
touches = calculate_labels(touches)

# Add labels to data frame
data = data.assign(label=touches.label)

In [83]:
data.head(20)

,Date,Time,Open,High,Low,Close,Vol,year,month,day,hour,minute,ticker,Time_tuple,threshold,t1,label
timestamp,,,,,,,,,,,,,,,,,
2021-11-02 03:07:00,2021-11-02,03:07,4597.25,4598.25,4597.25,4598.00,101.0,2021,11,2,03,07,MES,"(3, 7)",1.882271e-08,2021-11-02 03:22:00,-1.0
2021-11-02 03:08:00,2021-11-02,03:08,4598.00,4598.25,4597.50,4597.50,137.0,2021,11,2,03,08,MES,"(3, 8)",3.155678e-05,2021-11-02 03:23:00,1.0
2021-11-02 03:09:00,2021-11-02,03:09,4597.75,4598.00,4597.75,4598.00,49.0,2021,11,2,03,09,MES,"(3, 9)",3.140048e-05,2021-11-02 03:24:00,1.0
2021-11-02 03:10:00,2021-11-02,03:10,4598.00,4598.50,4597.75,4598.50,142.0,2021,11,2,03,10,MES,"(3, 10)",6.760163e-05,2021-11-02 03:25:00,-1.0
2021-11-02 03:11:00,2021-11-02,03:11,4598.25,4598.75,4597.75,4597.75,283.0,2021,11,2,03,11,MES,"(3, 11)",7.514171e-05,2021-11-02 03:26:00,-1.0
2021-11-02 03:12:00,2021-11-02,03:12,4597.75,4598.00,4597.75,4598.00,37.0,2021,11,2,03,12,MES,"(3, 12)",1.192424e-04,2021-11-02 03:27:00,-1.0
2021-11-02 03:13:00,2021-11-02,03:13,4597.75,4598.00,4597.50,4597.75,77.0,2021,11,2,03,13,MES,"(3, 13)",1.163407e-04,2021-11-02 03:28:00,-1.0
2021-11-02 03:14:00,2021-11-02,03:14,4597.75,4598.00,4597.75,4598.00,24.0,2021,11,2,03,14,MES,"(3, 14)",1.393885e-04,2021-11-02 03:29:00,-1.0
2021-11-02 03:15:00,2021-11-02,03:15,4598.00,4598.00,4597.25,4597.50,59.0,2021,11,2,03,15,MES,"(3, 15)",1.500574e-04,2021-11-02 03:30:00,-1.0


In [82]:
data['label'].value_counts()

 0.0    71317
-1.0    13161
 1.0    12471
Name: label, dtype: int64

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 97073 entries, 2021-11-02 01:03:00 to 2022-02-09 22:45:00
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        97073 non-null  datetime64[ns]
 1   Time        97073 non-null  object        
 2   Open        97073 non-null  float64       
 3   High        97073 non-null  float64       
 4   Low         97073 non-null  float64       
 5   Close       97073 non-null  float64       
 6   Vol         97073 non-null  float64       
 7   year        97073 non-null  int64         
 8   month       97073 non-null  int64         
 9   day         97073 non-null  int64         
 10  hour        97073 non-null  object        
 11  minute      97073 non-null  object        
 12  ticker      97073 non-null  object        
 13  Time_tuple  97073 non-null  object        
 14  threshold   97073 non-null  float64       
 15  t1          97073 non-null  datetim

In [74]:
data.head()

,Date,Time,Open,High,Low,Close,Vol,year,month,day,hour,minute,ticker,Time_tuple,threshold,t1,label
timestamp,,,,,,,,,,,,,,,,,
2021-11-02 03:07:00,2021-11-02,03:07,4597.25,4598.25,4597.25,4598.00,101.0,2021,11,2,03,07,MES,"(3, 7)",1.882271e-08,2021-11-02 03:22:00,-1.0
2021-11-02 03:08:00,2021-11-02,03:08,4598.00,4598.25,4597.50,4597.50,137.0,2021,11,2,03,08,MES,"(3, 8)",3.155678e-05,2021-11-02 03:23:00,1.0
2021-11-02 03:09:00,2021-11-02,03:09,4597.75,4598.00,4597.75,4598.00,49.0,2021,11,2,03,09,MES,"(3, 9)",3.140048e-05,2021-11-02 03:24:00,1.0
2021-11-02 03:10:00,2021-11-02,03:10,4598.00,4598.50,4597.75,4598.50,142.0,2021,11,2,03,10,MES,"(3, 10)",6.760163e-05,2021-11-02 03:25:00,-1.0
2021-11-02 03:11:00,2021-11-02,03:11,4598.25,4598.75,4597.75,4597.75,283.0,2021,11,2,03,11,MES,"(3, 11)",7.514171e-05,2021-11-02 03:26:00,-1.0


In [97]:
stock = 'ES'
file_name = 'Tripe_Barrier_Method_{}_{}_{}_ptsl_{}.parquet'.format(stock, 
                                                                 data['Date'].min().strftime("%Y-%m-%d"), 
                                                                    data['Date'].max().strftime("%Y-%m-%d"), 
                                                                        '-'.join([str(x) for x in ptSl] ) )

output_location_labels = os.path.join(input_folder_db, folder_path + stock,  'Tripe_Barrier_Method', file_name)


In [98]:
output_location_labels

'/mnt/Data/Tensor_Invest_Fund/data/Micro_Mini_Future_ES/Tripe_Barrier_Method/Tripe_Barrier_Method_ES_2021-11-02_2022-02-09_ptsl_1-1.parquet'

,Date,Time,Open,High,Low,Close,Vol,year,month,day,hour,minute,ticker,threshold,t1,label
timestamp,,,,,,,,,,,,,,,,
2021-11-02 03:07:00,2021-11-02,03:07,4597.25,4598.25,4597.25,4598.00,101.0,2021,11,2,03,07,MES,1.882271e-08,2021-11-02 03:22:00,-1.0
2021-11-02 03:08:00,2021-11-02,03:08,4598.00,4598.25,4597.50,4597.50,137.0,2021,11,2,03,08,MES,3.155678e-05,2021-11-02 03:23:00,1.0
2021-11-02 03:09:00,2021-11-02,03:09,4597.75,4598.00,4597.75,4598.00,49.0,2021,11,2,03,09,MES,3.140048e-05,2021-11-02 03:24:00,1.0
2021-11-02 03:10:00,2021-11-02,03:10,4598.00,4598.50,4597.75,4598.50,142.0,2021,11,2,03,10,MES,6.760163e-05,2021-11-02 03:25:00,-1.0
2021-11-02 03:11:00,2021-11-02,03:11,4598.25,4598.75,4597.75,4597.75,283.0,2021,11,2,03,11,MES,7.514171e-05,2021-11-02 03:26:00,-1.0


In [99]:
data.drop(columns = ['Time_tuple']).to_parquet(output_location_labels)

In [51]:
data = pd.read_parquet('/mnt/Data/Tensor_Invest_Fund/data/Micro_Mini_Future_ES/Tripe_Barrier_Method/Tripe_Barrier_Method_ES_2021-11-02_2022-02-09_ptsl_1-1.parquet')

In [52]:
data.reset_index(inplace = True)

In [81]:
def strategy_crossing_sma(_df, sma_w):
    """Golden cross occurs when short SMA crosses long SMA from below.
    Death cross is an opposite situation, when short SMA crosses long SMA from above.

    Args:
        _df (_type_): _description_
        sma_w (_type_): _description_
    """

    sma_short = sma_w[0]
    sma_long = sma_w[1]

    
    _df.loc[:,'s_sma_<_l_sma'] = pd.Series(np.where( _df[f'SMA_{sma_short}'] < _df[f'SMA_{sma_long}'], 1 , -1  ))
    _df.loc[:,'s_sma_<_l_sma_lag1'] = pd.Series(np.where( _df[f'SMA_{sma_short}'] < _df[f'SMA_{sma_long}'], 1 , -1  )).shift(1)
    
    _df.loc[:,'cross'] = np.where( _df['s_sma_<_l_sma'] != _df[f's_sma_<_l_sma_lag1'], 1 , 0  )
    _df.loc[:,'sma_cross_over'] = np.where( _df['cross'] == 0, 0, _df['cross'] * _df['s_sma_<_l_sma_lag1'])

    return _df


In [82]:
windows = [10,60]

for window in windows:

    data[f'SMA_{window}'] = data['Close'].rolling(window = window, closed = 'left').mean()

data = strategy_crossing_sma(data, sma_w = windows)

In [83]:
data[(data['sma_cross_over'] != 0)]

,timestamp,Date,Time,Open,High,Low,Close,Vol,year,month,...,ticker,threshold,t1,label,SMA_10,SMA_60,cross,s_sma_<_l_sma_lag1,sma_cross_over,s_sma_<_l_sma
0,2021-11-02 03:07:00,2021-11-02,03:07,4597.25,4598.25,4597.25,4598.00,101.0,2021,11,...,MES,1.882271e-08,2021-11-02 03:22:00,-1.0,NaN,NaN,1,NaN,NaN,-1
60,2021-11-02 04:07:00,2021-11-02,04:07,4592.50,4592.75,4592.00,4592.25,75.0,2021,11,...,MES,4.655117e-04,2021-11-02 04:22:00,0.0,4592.400,4594.250000,1,-1.0,-1.0,1
155,2021-11-02 05:42:00,2021-11-02,05:42,4589.25,4589.25,4589.00,4589.00,31.0,2021,11,...,MES,3.815280e-04,2021-11-02 05:57:00,0.0,4588.575,4588.558333,1,1.0,1.0,-1
228,2021-11-02 06:55:00,2021-11-02,06:55,4589.50,4589.50,4589.25,4589.50,65.0,2021,11,...,MES,4.559211e-04,2021-11-02 07:10:00,0.0,4589.925,4589.941667,1,-1.0,-1.0,1
249,2021-11-02 07:16:00,2021-11-02,07:16,4590.75,4591.00,4590.75,4590.75,37.0,2021,11,...,MES,3.821001e-04,2021-11-02 07:31:00,0.0,4589.900,4589.845833,1,1.0,1.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96691,2022-02-09 18:28:00,2022-02-09,18:28,4567.00,4567.50,4566.00,4566.50,1216.0,2022,2,...,MES,1.257140e-03,2022-02-09 18:43:00,0.0,4565.550,4565.125000,1,1.0,1.0,-1
96744,2022-02-09 19:21:00,2022-02-09,19:21,4568.75,4569.75,4568.00,4569.75,927.0,2022,2,...,MES,1.077319e-03,2022-02-09 19:36:00,0.0,4568.125,4568.237500,1,-1.0,-1.0,1
96748,2022-02-09 19:25:00,2022-02-09,19:25,4570.00,4570.00,4568.00,4568.50,860.0,2022,2,...,MES,1.039926e-03,2022-02-09 19:40:00,0.0,4568.550,4568.520833,1,1.0,1.0,-1
96840,2022-02-09 20:57:00,2022-02-09,20:57,4575.50,4575.50,4573.50,4574.25,2043.0,2022,2,...,MES,7.773537e-04,2022-02-09 21:12:00,-1.0,4576.750,4576.975000,1,-1.0,-1.0,1


In [84]:
data[(data['timestamp'] >= '2021-11-02 07:05:00')].head(30)

,timestamp,Date,Time,Open,High,Low,Close,Vol,year,month,...,ticker,threshold,t1,label,SMA_10,SMA_60,cross,s_sma_<_l_sma_lag1,sma_cross_over,s_sma_<_l_sma
238,2021-11-02 07:05:00,2021-11-02,07:05,4588.50,4588.75,4588.50,4588.75,67.0,2021,11,...,MES,0.000423,2021-11-02 07:20:00,1.0,4589.225,4589.929167,0,1.0,0.0,1
239,2021-11-02 07:06:00,2021-11-02,07:06,4588.50,4589.25,4588.50,4589.25,125.0,2021,11,...,MES,0.000419,2021-11-02 07:21:00,0.0,4589.150,4589.912500,0,1.0,0.0,1
240,2021-11-02 07:07:00,2021-11-02,07:07,4589.25,4589.25,4589.00,4589.00,59.0,2021,11,...,MES,0.000416,2021-11-02 07:22:00,0.0,4589.125,4589.904167,0,1.0,0.0,1
241,2021-11-02 07:08:00,2021-11-02,07:08,4589.25,4589.75,4589.25,4589.75,110.0,2021,11,...,MES,0.000412,2021-11-02 07:23:00,0.0,4589.075,4589.887500,0,1.0,0.0,1
242,2021-11-02 07:09:00,2021-11-02,07:09,4589.50,4590.25,4589.50,4590.00,97.0,2021,11,...,MES,0.000408,2021-11-02 07:24:00,0.0,4589.075,4589.879167,0,1.0,0.0,1
243,2021-11-02 07:10:00,2021-11-02,07:10,4589.75,4590.00,4589.50,4590.00,31.0,2021,11,...,MES,0.000404,2021-11-02 07:25:00,0.0,4589.125,4589.879167,0,1.0,0.0,1
244,2021-11-02 07:11:00,2021-11-02,07:11,4590.00,4590.00,4589.75,4589.75,32.0,2021,11,...,MES,0.000400,2021-11-02 07:26:00,0.0,4589.200,4589.879167,0,1.0,0.0,1
245,2021-11-02 07:12:00,2021-11-02,07:12,4589.75,4590.00,4589.75,4590.00,34.0,2021,11,...,MES,0.000397,2021-11-02 07:27:00,0.0,4589.275,4589.870833,0,1.0,0.0,1
246,2021-11-02 07:13:00,2021-11-02,07:13,4590.00,4590.50,4590.00,4590.25,65.0,2021,11,...,MES,0.000393,2021-11-02 07:28:00,0.0,4589.350,4589.858333,0,1.0,0.0,1
247,2021-11-02 07:14:00,2021-11-02,07:14,4590.25,4590.75,4590.25,4590.25,58.0,2021,11,...,MES,0.000390,2021-11-02 07:29:00,0.0,4589.550,4589.850000,0,1.0,0.0,1


In [147]:
data['metalabel'] =  get_metalabels(y_model1 = data['sma_cross_over'] , y_true = data['label'])



In [148]:
data['metalabel'].value_counts()

0.0    96605
1.0      344
Name: metalabel, dtype: int64

## 2. Synthetic Minority Oversampling Technique

In [149]:
from imblearn.over_sampling import SMOTE

# Ref: https://arxiv.org/pdf/1106.1813.pdf

In [150]:
#data.set_index(['timestamp'], inplace = True)

In [152]:
_df = data.copy(deep = True).dropna()

model_feat = [ 'Open', 'High', 'Low', 'Close', 'Vol',
       'year', 'month', 'day', 'hour', 'minute','SMA_10', 'SMA_60','sma_cross_over',]

train_cutoff = datetime(2022, 1, 15, 0,0,0)

X = _df.filter(model_feat)

y = np.squeeze(_df['metalabel'])

X_train, y_train = X[X.index < train_cutoff], y[y.index < train_cutoff]
X_test, y_test = X[X.index >= train_cutoff], y[y.index >= train_cutoff]

sm = SMOTE()

X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

In [153]:
values, counts =  np.unique(y_train_res, return_counts=True)

In [154]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train_res, y_train_res)
y_pred = clf.predict(X_test)

In [155]:
np.unique(y_pred, return_counts=True)

(array([0., 1.]), array([16801,  7784]))

In [160]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
cm= confusion_matrix(y_test, y_pred)

In [157]:
cm

array([[16745,  7743],
       [   56,    41]])

0.41075834469839423

In [163]:
# **Precision**: TP/ (TP + FP)
# **Recall**: TP/ (TP + FN)

# Model 2
# Target is a high precision reduce FP

# Model1 
# Target is high recall reduce FN

precision = precision_score(y_test , y_pred )
recall = recall_score(y_test , y_pred )
f1 = f1_score(y_test , y_pred , average='macro')

print("Precision is",  precision)
print("Recall is:", recall)
print("F1 score is:", f1)

Precision is 0.0052672147995889
Recall is: 0.422680412371134
F1 score is: 0.41075834469839423


0.422680412371134

## Data Storage

../../data/Model1\MNQ\Bar_Entry_Positions_2021-06-01_2021-10-14_t10-r10_w5.parquet


In [16]:
print("data Stored at:")

print(" Classification entry data: ", output_location_entry)

print(" Windows classification:", output_location_windows)

data Stored at:
 Classification entry data:  ../../data/Model1\MNQ\Bar_Entry_Positions_2021-06-01_2021-10-14_t10-r10_w5.parquet
 Windows classification: ../../data/Model1\MNQ\Windows_Price_Region_2021-06-01_2021-10-14_t10-r10_w5.parquet


In [17]:
output_location_entry

'../../data/Model1\\MNQ\\Bar_Entry_Positions_2021-06-01_2021-10-14_t10-r10_w5.parquet'